# Modelo: Bagging con KNN

El siguiente modelo que vamos a entrenar se trata de un ensamble. En particular usaremos bagging junto con KNN. Utilizaremos BagginClassifier, KNeighborsClassifier, distintos scalers y métricas de la libreria sklearn

## Librerias y funciones necesarias

Comenzamos importando todas las librerias y funciones necesarias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,Normalizer,RobustScaler,StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier

In [2]:
from preprocessing import obtener_datasets
from preprocessing import aplicar_preparacion
from preprocessing import conversion_numerica
from preprocessing import plot_roc_curves
from preprocessing import graficar_matriz_confusion
from preprocessing import aplicar_preparacion_generalizado
from preprocessing import conversion_numerica_generalizada
from preprocessing import get_dataframe_scaled
from preprocessing import reduccion_rfecv
from preprocessing import get_dataframe_polynomial

## Primer preprocesamiento

En primer lugar obtenemos el dataset para entrenar y el holdout. En segundo lugar, aplicamos una función que trabaja sobre las features, generalizando algunas y dejando de lado otras según lo observado en la primer parte de este trabajo práctico. También separamos a la variable target del resto del dataset. También, convertimos a númericas las variables categoricas para poder entrenar nuestro modelo. Por último, vamos a escalar nuestro dataset con StandarScaler de sklearn

In [3]:
df, df_holdout = obtener_datasets()
X_df, y_df = aplicar_preparacion(df)
X_df = conversion_numerica(X_df) 

Aplicando 'conversion_numerica' en las variables categóricas.


Luego vamos a realizar un split del dataset para dividir en train y test. Como observamos en la primer parte de este trabajo práctico, la variable target no esta distribuida uniformente por lo cual realizamos una división estratificada

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=30,stratify=y_df)

In [5]:
scaler = StandardScaler()
X_train = get_dataframe_scaled(X_train,scaler)
X_test = get_dataframe_scaled(X_test,scaler)

### Entrenamiento

#### Hiperparámetros de KNN

En primer lugar vamos a buscar hiperparámetros para KNN con este preprocessing. Utilizamos Gridsearch de sklearn

In [6]:
parametros = {"n_neighbors":np.arange(20,50,5),"weights":["uniform","distance"]}
clf = KNeighborsClassifier()
cv = StratifiedKFold(n_splits=5,shuffle= True, random_state= 10).split(X_train, y_train)
clf = GridSearchCV(clf, parametros, scoring='roc_auc', cv=cv, n_jobs = -1, verbose=4)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.7min finished


GridSearchCV(cv=<generator object _BaseKFold.split at 0x7f2bd2f3af90>,
             estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': array([20, 25, 30, 35, 40, 45]),
                         'weights': ['uniform', 'distance']},
             scoring='roc_auc', verbose=4)

Veamos que hiperprámetros usar

In [7]:
clf.best_params_

{'n_neighbors': 40, 'weights': 'uniform'}

#### Bagging

Ahora si configuramos Bagging. Luego utilizamos GridSearch nuevamente para la busqueda de hiperparámetros

In [12]:
clf_bagging = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=40,weights='uniform'))
cv = StratifiedKFold(n_splits=5,shuffle= True, random_state= 10).split(X_train, y_train)
parametros = parametros = {"n_estimators":np.arange(50,60,40)}
clf_bagging = GridSearchCV(clf_bagging, parametros, scoring='roc_auc', cv=cv, n_jobs = -1, verbose=4)
clf_bagging.fit(X_train, y_train)
y_pred = clf_bagging.predict(X_test)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 19.9min remaining: 29.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 27.6min finished


#### Métricas

In [13]:
print("AUC-ROC score sobre test: ", "%0.16f"  % roc_auc_score(y_test, clf_bagging.predict_proba(X_test)[:, 1]))
print("AUC-ROC score sobre train: ", "%0.16f"  % roc_auc_score(y_train, clf_bagging.predict_proba(X_train)[:, 1]))
print("Accuracy sobre test: ", "%0.16f"  % accuracy_score(y_pred, y_test))
print("Los mejores hiperpametros elegidos: ", clf_bagging.best_params_)
print(classification_report(y_pred, y_test, target_names=["Bajo valor", "Alto valor"]))
graficar_matriz_confusion(y_test, y_pred)
plot_roc_curves(clf_bagging, X_test, y_test, X_train, y_train)

AUC-ROC score sobre test:  0.8844697282350756


KeyboardInterrupt: 

## Segundo preprocesamiento

En primer lugar obtenemos el dataset para entrenar y el holdout. En segundo lugar, aplicamos una función que trabaja sobre las features, generalizando algunas y dejando de lado otras según lo observado en la primer parte de este trabajo práctico. También separamos a la variable target del resto del dataset. También, convertimos a númericas las variables categoricas para poder entrenar nuestro modelo. Por último, vamos a escalar nuestro dataset con Normalizer de sklearn

In [ ]:
df, df_holdout = obtener_datasets()
X_df, y_df = aplicar_preparacion(df)
X_df = conversion_numerica(X_df) 

Luego vamos a realizar un split del dataset para dividir en train y test. Como observamos en la primer parte de este trabajo práctico, la variable target no esta distribuida uniformente por lo cual realizamos una división estratificada

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=30,stratify=y_df)

In [ ]:
scaler = Normalizer()
X_train = get_dataframe_scaled(X_train,scaler)
X_test = get_dataframe_scaled(X_test,scaler)

### Entrenamiento

#### Hiperparámetros de KNN

En primer lugar vamos a buscar hiperparámetros para KNN con este preprocessing. Utilizamos Gridsearch de sklearn

In [ ]:
parametros = {"n_neighbors":np.arange(20,50,5),"weights":["uniform","distance"]}
clf = KNeighborsClassifier()
cv = StratifiedKFold(n_splits=5,shuffle= True, random_state= 10).split(X_train, y_train)
clf = GridSearchCV(clf, parametros, scoring='roc_auc', cv=cv, n_jobs = -1, verbose=4)
clf.fit(X_train, y_train)

Veamos que hiperprámetros usar

In [ ]:
clf.best_params_

#### Bagging

Ahora si configuramos Bagging. Luego utilizamos GridSearch nuevamente para la busqueda de hiperparámetros

In [ ]:
clf_bagging = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=45,weights='uniform'))
cv = StratifiedKFold(n_splits=5,shuffle= True, random_state= 10).split(X_train, y_train)
parametros = parametros = {"n_estimators":np.arange(50,60,40)}
clf_bagging = GridSearchCV(clf_bagging, parametros, scoring='roc_auc', cv=cv, n_jobs = -1, verbose=4)
clf_bagging.fit(X_train, y_train)
y_pred = clf_bagging.predict(X_test)

#### Métricas

In [ ]:
print("AUC-ROC score sobre test: ", "%0.16f"  % roc_auc_score(y_test, clf_bagging.predict_proba(X_test)[:, 1]))
print("AUC-ROC score sobre train: ", "%0.16f"  % roc_auc_score(y_train, clf_bagging.predict_proba(X_train)[:, 1]))
print("Accuracy sobre test: ", "%0.16f"  % accuracy_score(y_pred, y_test))
print("Los mejores hiperpametros elegidos: ", clf_bagging.best_params_)
print(classification_report(y_pred, y_test, target_names=["Bajo valor", "Alto valor"]))
graficar_matriz_confusion(y_test, y_pred)
plot_roc_curves(clf_bagging, X_test, y_test, X_train, y_train)
